# New notebook to keep dev clean

In [255]:
import os
import sys
import praw
from praw import Reddit

class WallpaperScraper:
    reddit_oath = {
        'client_id': 'pIxpnAoiGfwE-g',
        'client_secret': 'Ueccpv4dJegXUYbmAIdwxevxjDs',
        'user_agent': 'wallpapers'
    }
    
    imgur_oath = {
        'client_id': '29b27fc1363aa92',
        'client_secret': 'ef58b651b764dfb487809e64a2d62982e1a6392e'
    }
    
    imgur_headers = {
      'Authorization': f'Client-ID {imgur_oath["client_id"]}'
    }
    
#     sort_options = {'controversial', 'gilded', 'hot', 'new', 'rising', 'top'}
#     time_filter = {'all', 'day', 'hour', 'month', 'week', 'year'}
    
    def __init__(self, img_dir, sort='top', time_filter='week'):
        """
        Initialize a wallpaper subreddit scraper instance with feed options.
        """
        
        self._img_dir = img_dir
        self._mk_img_dir()
        
        self._wallpapers = Reddit(**self.reddit_oath).subreddit('wallpapers')
        if sort in {'controversial', 'top'}:
            self._wallpapers = getattr(self._wallpapers, sort)(limit=1000, time_filter=time_filter)
        else:
            self._wallpapers = getattr(self._wallpapers, sort)(limit=1000)
        
    def _mk_img_dir(self):
        """
        Checks and initializes a directory for image files to be downloaded.
        """
        if not os.path.exists(self._img_dir):
            os.mkdir(self._img_dir)
        if not os.path.exists(os.path.join(self._img_dir, 'buffer')):
            os.mkdir(os.path.join(self._img_dir, 'buffer'))
            
    def _img_url_extract(self, img_sub):
        """
        Extract and return the url hyperlink of only PRAW's submission-type objects.
        """
        if isinstance(img_sub, praw.models.reddit.submission.Submission):
            return img_sub.url
        return None

In [241]:
test = WallpaperScraper('wallpapers', 'top', 'year')

In [84]:
for idx, img_sub in enumerate(test._wallpapers):
    if isinstance(img_sub, praw.models.reddit.submission.Submission):
        img_sub.url.split('.')[-1]
        file_ext = img_sub.url.split('.')[-1]
        print(img_sub.url, file_ext, img_sub.thumbnail)
    #     with open(f'tmp/tmp{idx}.{file_ext}', 'wb') as tmp:
    #         r = requests.get(img_sub.url, stream=True)
    #         if r.ok:
    #             tmp.write(r.content)
    #     file_ext = img_sub.thumbnail.split('.')[-1]
    #     with open(f'tmp/tmp{idx}_.{file_ext}', 'wb') as tmp:
    #         r = requests.get(img_sub.thumbnail, stream=True)
    #         if r.ok:
    #             tmp.write(r.content)
    
    if idx == 10:
        break

https://imgur.com/a/22D4xOJ com/a/22D4xOJ https://b.thumbs.redditmedia.com/SSWGbJdT06dgxC7DPRUBMy2FjzV_JfwovqVVISHdgjI.jpg
https://i.redd.it/o1vmuo8vend21.jpg jpg https://b.thumbs.redditmedia.com/e8B7NrIZktnovKk-92O2KTZrXm9_TvSG8oJz3CgYjiI.jpg
https://i.redd.it/urs703bti4f31.jpg jpg https://b.thumbs.redditmedia.com/gTMLYkASEJ-AUwROYXGDx2pjAWz1NcBJCftzDTpXtkU.jpg
https://i.redd.it/w3j6kry5mvu21.jpg jpg https://a.thumbs.redditmedia.com/_nnn6idTvszlVdR9X9YT26zRkGsucCLSb-_jLCdbC74.jpg
https://i.redd.it/dc3m37tcbdj31.jpg jpg https://b.thumbs.redditmedia.com/KEKE9kIJ9-qK-GduhhggVD1W9T1G5GFrKm6oF0pwprw.jpg
https://i.redd.it/bmp9m1mrzyg31.jpg jpg https://b.thumbs.redditmedia.com/-lUkWa9M9BmewRGV2HzaG6-xBtzPWXjdwIGJnPIjWMs.jpg
https://i.redd.it/5fcdmvjfzmy21.jpg jpg https://b.thumbs.redditmedia.com/Y_Zo2ch29bXxsKh1n8CGJORAytnHnYT32rbaGbWu8cg.jpg
https://i.redd.it/75fvuwxirjy21.jpg jpg https://b.thumbs.redditmedia.com/KbcC0ccUjB77SMZ7A5Y-gh3BBG2Evolx8_dlwnrlsGM.jpg
https://i.redd.it/hne6x6uo84h3

# Need to develop a class - image generator

- Scrape only scrapable images
    - Predictable scrapes: e.g. imgur/reddit
        - Reddit API provides a thumbnail!! - Great for speed-up of image processing and IO
    - Unpredictable scrapes: Other sites/reddit comments
- Develop a pre-buffer
    - Image pops are quick, but buffer can buffer in the background
    - Store image in buffer folder
    > 1 - URL Buffer
    
    > 2 - Image Buffer

In [82]:
def _img_url_extract(img_sub):
    """
    Extract and return the url hyperlink of only PRAW's submission-type objects.
    Include the link to a thumbnail if it exists
    """
    if isinstance(img_sub, praw.models.reddit.submission.Submission):
        return (img_sub.url, img_sub.thumbnail)
    return None

In [89]:
def is_submission(img_sub):
    return isinstance(img_sub, praw.models.reddit.submission.Submission)

In [90]:
def is_img(url):
    return url.split('.')[-1] in {'jpg','png'}

In [94]:
def is_imgur(url):
    return 'imgur' in url

In [234]:
imgur_oath = {
    'client_id': '29b27fc1363aa92',
    'client_secret': 'ef58b651b764dfb487809e64a2d62982e1a6392e'
}

imgur_headers = {
    'Authorization': f'Client-ID {imgur_oath["client_id"]}'
}

def get_imgur_img(url):
    img_urls = []
    imgur_hash = url.strip('/').split('/')[-1].split('?')[0]
    
    # Try album endpoint API
    api_url = f'https://api.imgur.com/3/album/{imgur_hash}/images'
    response = requests.request('GET', api_url, headers = imgur_headers, allow_redirects=False)
    if response.ok:
        img_urls += [(image['link'], None) for image in json.loads(response.text)['data']]
    
    else:
        # Try gallery endpoint API
        api_url = f'https://api.imgur.com/3/gallery/album/{gallery_hash}'
        response = requests.request('GET', api_url, headers = imgur_headers, allow_redirects=False)
        if response.ok:
            img_urls += [(image['link'], None) for image in json.loads(response.text)['data']['images']]

    if len(img_urls) == 0:
        print(img_url, 'ERROROROROROROROROROROR')
        
    return img_urls

In [202]:
get_imgur_img('https://imgur.com/a/WepxW')

[('https://i.imgur.com/dWxfPoG.png', None),
 ('https://i.imgur.com/woEnEXG.png', None),
 ('https://i.imgur.com/rihdXYC.png', None),
 ('https://i.imgur.com/eYQsah5.png', None),
 ('https://i.imgur.com/MQWRu8C.png', None)]

In [203]:
get_imgur_img('https://imgur.com/a/rSZlZ')

[('https://i.imgur.com/NVQ1nuu.png', None),
 ('https://i.imgur.com/rDQ6BEA.jpg', None),
 ('https://i.imgur.com/gIMtb0Z.png', None),
 ('https://i.imgur.com/GeZmbJA.png', None),
 ('https://i.imgur.com/AGWHnX0.jpg', None),
 ('https://i.imgur.com/yt86fMB.jpg', None),
 ('https://i.imgur.com/cG9eXMk.jpg', None),
 ('https://i.imgur.com/4gdcscj.jpg', None),
 ('https://i.imgur.com/uA1NSv3.png', None)]

In [204]:
get_imgur_img('https://imgur.com/a/22D4xOJ')

[('https://i.imgur.com/kQqTw9u.png', None),
 ('https://i.imgur.com/kHDJ6RJ.png', None),
 ('https://i.imgur.com/Tbk5Dbz.png', None),
 ('https://i.imgur.com/KOq3LNx.png', None),
 ('https://i.imgur.com/Sb4Lmwq.png', None),
 ('https://i.imgur.com/yw9QhM2.png', None),
 ('https://i.imgur.com/UrYIUCw.png', None),
 ('https://i.imgur.com/3u4IeYb.png', None)]

In [256]:
test = WallpaperScraper('wallpapers', 'top', 'year')

In [257]:
import requests
import json

urls = []
non_reddit = 0
for idx, img_sub in enumerate(test._wallpapers):
    if is_submission(img_sub):
        if is_img(img_sub.url):
            urls.append((img_sub.url, img_sub.thumbnail))
        elif is_imgur(img_sub.url):
            urls += get_imgur_img(img_sub.url)
        else:
            non_reddit += 1
    
    if idx == 500:
        break

In [258]:
len(urls)

529

In [259]:
non_reddit

6

In [260]:
urls

[('https://i.redd.it/llud0j7a7ch21.jpg',
  'https://b.thumbs.redditmedia.com/ckxpzbssft-G8Pc59doYKPse-TOs_lTAE1kQrYeMJeg.jpg'),
 ('https://i.redd.it/qqj4tnhc2qh21.png',
  'https://b.thumbs.redditmedia.com/tBFAVX8vmAbO5HcHgpGNVuuFsls3e_n4ud5bW8QsqxM.jpg'),
 ('https://i.redd.it/t6gatr97skp31.png',
  'https://b.thumbs.redditmedia.com/gN62aY2dgLFEZVPrN2bxD76s1c-LefxMQbYUHZoK4uU.jpg'),
 ('https://i.imgur.com/PkVweyp.jpg',
  'https://b.thumbs.redditmedia.com/18lm6Jl5tawsJqvf37lgNRVEUR99IBvRhmE9hXxXPkc.jpg'),
 ('https://i.imgur.com/kE2yyYK.jpg',
  'https://b.thumbs.redditmedia.com/uxWga0wF2OqTPQwcHv0HB-PkZkbRkhihINxXdJGv2QQ.jpg'),
 ('https://i.redd.it/d3h7ofnt73621.png',
  'https://b.thumbs.redditmedia.com/Rh-uNIFdF58iVWMFaLFb4lS4eUTeyXwZgGEvti8PKfQ.jpg'),
 ('https://i.imgur.com/mrOs9ZC.png',
  'https://b.thumbs.redditmedia.com/g9X4wFmBblyEQ3QFR6BspjYFFXiy9HK-ydXUdpzjoCQ.jpg'),
 ('https://i.imgur.com/X2qDwdN.jpg',
  'https://b.thumbs.redditmedia.com/BaclZQ1CpfBqGvhk16TwWtCTlGW5YmFCsLVGACiFNwA